# Nevada Scraping 

In [1]:
# set up 
import pandas as pd 
import requests 
import bs4
import numpy as np
import time 
import re
from tika import parser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from io import BytesIO
from urllib.request import urlopen
from selenium.webdriver.chrome.options import Options

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

%xmode Minimal

Exception reporting mode: Minimal


In [2]:
# # set headers
# header_list = {'User-Agent': REDACTED,
#               'Accept-Language': 'en-US,en;q=0.9'}

In [3]:
# state - UPDATE EACH TIME
state = 'nevada' 

In [ ]:
# review of robots.txt - no restrictions

In [ ]:
# test = pd.read_csv(f'{state}_2023.csv')
# test.head()

## Scrape links 
`Selenium`

In [14]:
driver = webdriver.Chrome()
driver.get('https://gov.nv.gov/Newsroom/PRs/news-releases/')

wait = WebDriverWait(driver, 3)
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'm-4')))

section = driver.find_element(By.XPATH, '//*[@id="content_main"]')
results = section.find_elements(By.CLASS_NAME, 'm-4')
# results

results_list = []

for i in results:
    try: 
        results_dic = {'title': i.find_element(By.TAG_NAME, 'a').text,
                       'date': i.find_element(By.TAG_NAME, 'p').text,
                       'link': i.find_element(By.TAG_NAME, 'a').get_attribute('href')}
        results_list.append(results_dic)
    except:
        pass
          
df = pd.DataFrame(results_list)

driver.close()

[<selenium.webdriver.remote.webelement.WebElement (session="d6905d71177e8e9848cf47468fdb793c", element="02D0067DADC2D9E33D32035E24C72495_element_13")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d6905d71177e8e9848cf47468fdb793c", element="02D0067DADC2D9E33D32035E24C72495_element_14")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d6905d71177e8e9848cf47468fdb793c", element="02D0067DADC2D9E33D32035E24C72495_element_15")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d6905d71177e8e9848cf47468fdb793c", element="02D0067DADC2D9E33D32035E24C72495_element_16")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d6905d71177e8e9848cf47468fdb793c", element="02D0067DADC2D9E33D32035E24C72495_element_17")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d6905d71177e8e9848cf47468fdb793c", element="02D0067DADC2D9E33D32035E24C72495_element_18")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d6905d71177e8e9848cf47468fdb

In [16]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   91 non-null     object
 1   date    91 non-null     object
 2   link    91 non-null     object
dtypes: object(3)
memory usage: 2.3+ KB


,title,date,link
0,Governor Joe Lombardo Appoints Dr. Vicki Mazzo...,"February 14, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-14-0...
1,press@gov.nv.gov,Communications Director\nElizabeth Ray\npress@...,mailto:press@gov.nv.gov
2,Governor Joe Lombardo and Controller Andy Matt...,"February 13, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-1...
3,Media Advisory: Controller Andy Matthews To An...,"February 9th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-0...
4,DMV Director Announces Retirement,"February 9th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-0...


In [17]:
# drop the non-article
df = df.drop([1])
df.head()

,title,date,link
0,Governor Joe Lombardo Appoints Dr. Vicki Mazzo...,"February 14, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-14-0...
2,Governor Joe Lombardo and Controller Andy Matt...,"February 13, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-1...
3,Media Advisory: Controller Andy Matthews To An...,"February 9th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-0...
4,DMV Director Announces Retirement,"February 9th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-0...
5,ICYMI: Governor Joe Lombardo Announces Over 14...,"January 25th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-01-2...


In [19]:
# add an empty content column
df['content'] = np.nan
df.head()

,title,date,link,content
0,Governor Joe Lombardo Appoints Dr. Vicki Mazzo...,"February 14, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-14-0...,NaN
2,Governor Joe Lombardo and Controller Andy Matt...,"February 13, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-1...,NaN
3,Media Advisory: Controller Andy Matthews To An...,"February 9th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-0...,NaN
4,DMV Director Announces Retirement,"February 9th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-0...,NaN
5,ICYMI: Governor Joe Lombardo Announces Over 14...,"January 25th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-01-2...,NaN


In [20]:
# save
df.to_csv(f'{state}_links.csv', index = False)

In [21]:
test = pd.read_csv(f'{state}_links.csv')
test.head()

,title,date,link,content
0,Governor Joe Lombardo Appoints Dr. Vicki Mazzo...,"February 14, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-14-0...,NaN
1,Governor Joe Lombardo and Controller Andy Matt...,"February 13, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-1...,NaN
2,Media Advisory: Controller Andy Matthews To An...,"February 9th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-0...,NaN
3,DMV Director Announces Retirement,"February 9th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-0...,NaN
4,ICYMI: Governor Joe Lombardo Announces Over 14...,"January 25th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-01-2...,NaN


## Scrape content from links 
`Beautiful Soup`

### Test with one link

In [22]:
# test with one
link = 'https://gov.nv.gov/Newsroom/PRs/2024/2024-01-25_newjobs/'
server_response = requests.get(link, headers = header_list)
server_response

<Response [200]>

In [27]:
# test with one contd 
soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
result = soup_link.find('div', class_ = 'col-md-9').find_all('p')
# result
string_result = [str(i.text) for i in result]
# string_result
string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean]
result_merged = " ".join(string_result_clean)
result_merged

# #p_results[3]
# p_text = [str(i.text) for i in p_results[3]]
# p_text
# string_result = [str(i.text) for i in result]
# string_result
# string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
# string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean]
# string_result_clean = [i.replace(u'\r', u'') for i in string_result_clean]
# result_merged = " ".join(string_result_clean)
# result_merged

'Today, Governor Joe Lombardo and the Nevada Governor’s Office of Economic Development (GOED) approved three companies that will receive $14,520,823 in tax abatements. In return, these companies are projected to create 148 jobs in the next two years at an average weighted hourly wage of $37.49. That figure is expected to grow to 219 jobs in five years. Additionally, these companies will make a capital equipment investment of $120,717,140 and generate $37,536,026 in new tax revenues over the 10-year abatement period. “These companies are making a major investment and creating exceptional jobs for our state,” said Governor Joe Lombardo. “These three companies alone will invest more than $120 million in Nevada, paying an average wage of $37.49. Since I took office, 28 companies have been assisted and are projected to create 5, 427 jobs in their first five years of operation. They have invested more than $4.1 billion in capital equipment and will generate more than $1 billion in new tax re

In [ ]:
# define link scraping 
def get_nevada_content(link, df, header_list): 
    try: 
        # check if content has already been pulled
        result = df.loc[df.link == link, 'content'].notnull().all()
        if result:
            return f'content already existing for {link}' 
        else: 
            try: 
                server_response = requests.get(link, headers = header_list)
                soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
                result = soup_link.find('div', class_ = 'col-md-9').find_all('p')
                string_result = [str(i.text) for i in result]
                string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
                string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean]
                result_merged = " ".join(string_result_clean)
                
                df.loc[df.link == link, 'content'] = result_merged
                return f"success: content added from {link}"
            except: 
                pass
    except Exception as e: 
        return f"unable to gather content from {link} due to {e}"
    #return df_updated 

In [40]:
get_nevada_content(combined.link[0], combined, header_list) 
combined.content[0]

'content already existing for https://gov.nv.gov/Newsroom/PRs/2024/2024-14-02_cannabis-compliance-board-appointment/'

'Today, Governor Joe Lombardo announced his appointment of Dr. Vicki Mazzorana to the Cannabis Compliance Board. “I’m honored to be appointed to the Cannabis Compliance Board,” said Dr. Vicki Mazzorana. “I believe in responsible and reasonable cannabis regulation, and I look forward to utilizing my unique background in emergency medicine to benefit the evolving cannabis industry in Nevada.” Mazzorana earned her undergraduate degree in biology from the University of Illinois, her M.D. from Rush Medical College, and her M.B.A. from Louisiana State University. She received training at UC Davis in General Surgery and Emergency Medicine at the Medical College of Wisconsin. Mazzorana has served as an emergency physician for nearly three decades, and she has served as a ringside physician for the Nevada State Athletic Commission since 2008. Additionally, Mazzorana serves on both the National Disaster Medical Team, as well as on Nevada’s Search and Rescue Team. The Cannabis Compliance Board wa

### Scrape content from all links

In [41]:
for i in combined.link:
     get_nevada_content(i, combined, header_list) 

'content already existing for https://gov.nv.gov/Newsroom/PRs/2024/2024-14-02_cannabis-compliance-board-appointment/'

'success: content added from https://gov.nv.gov/Newsroom/PRs/2024/2024-02-13_new-open-finance-portal/'

'success: content added from https://gov.nv.gov/Newsroom/PRs/2024/2024-02-09_open-finance-portal/'

'success: content added from https://gov.nv.gov/Newsroom/PRs/2024/2024-02-09_dmv-director-retirement/'

'success: content added from https://gov.nv.gov/Newsroom/PRs/2024/2024-01-25_newjobs/'

...

'success: content added from https://gov.nv.gov/Newsroom/PRs/2023/2023-01-23_StateOfTheStateAddress/'

'success: content added from https://gov.nv.gov/Newsroom/PRs/2023/2023-01-12_Executive_Orders/'

'success: content added from https://gov.nv.gov/Newsroom/PRs/2023/2023-01-11_Kirk_Hendrick/'

'success: content added from https://gov.nv.gov/Newsroom/PRs/2023/2023-01-10_StateOffceClosure/'

'success: content added from https://gov.nv.gov/Newsroom/PRs/2023/2023-01-06_PropaneProclamation/'

In [42]:
combined.head()

,title,date,link,content
0,Governor Joe Lombardo Appoints Dr. Vicki Mazzo...,"February 14, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-14-0...,"Today, Governor Joe Lombardo announced his app..."
1,Governor Joe Lombardo and Controller Andy Matt...,"February 13, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-1...,"Today, Governor Joe Lombardo and Controller An..."
2,Media Advisory: Controller Andy Matthews To An...,"February 9th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-0...,FOR PLANNING PURPOSES ONLY FOR PLANNING PURPOS...
3,DMV Director Announces Retirement,"February 9th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-02-0...,After 30 years of public service to the State ...
4,ICYMI: Governor Joe Lombardo Announces Over 14...,"January 25th, 2024",https://gov.nv.gov/Newsroom/PRs/2024/2024-01-2...,"Today, Governor Joe Lombardo and the Nevada Go..."


## Validate and Clean

In [43]:
#check that content was pulled accurately
combined[combined.content.isna()]

,title,date,link,content
64,Governor Lombardo Amends Emergency Declaration...,"March 11, 2023",https://gov.nv.gov/xxx,NaN
80,Governor Lombardo Appoints George Assad to Nev...,"January 30, 2023",https://gov.nv.gov/xxx,NaN
89,Governor Lombardo Issues Winter Weather Update...,"January 4, 2023",https://gov.nv.gov/xxx,NaN


In [44]:
# manual review shows that links are broken/not available so we'll drop these
combined = combined.drop([64,80,89])
combined[combined.content.isna()]

,title,date,link,content


In [48]:
# subset to only 2023 dates
combined['date_clean'] = pd.to_datetime(combined['date'])

combined_2023 = combined[(combined['date_clean'] >= '2023-01-01')
                     & (combined['date_clean'] <= '2023-12-31')].copy()

combined_2023 = combined_2023.drop(columns=['date']).copy()

In [49]:
combined_2023.head()
combined_2023.info()

,title,link,content,date_clean
7,Governor Lombardo Announces Chief of Staff Tra...,https://gov.nv.gov/Newsroom/PRs/2023/2023-12-2...,"CARSON CITY, NV – Governor Joe Lombardo today ...",2023-12-27
8,Governor Lombardo Announces Las Vegas Stadium ...,https://gov.nv.gov/Newsroom/PRs/2023/2023-12-0...,"Today, Governor Joe Lombardo announced his app...",2023-12-05
9,Governor Lombardo Announces James Humm as Exec...,https://gov.nv.gov/Newsroom/PRs/2023/2023-12-0...,"Today, Governor Joe Lombardo announced James “...",2023-12-05
10,Governor Lombardo Lowers Flags to Half-Staff i...,https://gov.nv.gov/Newsroom/PRs/2023/2023-11-3...,"Today, Governor Joe Lombardo announced that th...",2023-11-30
11,Governor Lombardo Statement on Nevada State Tr...,https://gov.nv.gov/Newsroom/PRs/2023/2023-11-3...,Following the tragic report from Nevada State ...,2023-11-30


<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 7 to 88
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   title       80 non-null     object        
 1   link        80 non-null     object        
 2   content     80 non-null     object        
 3   date_clean  80 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 3.1+ KB


In [50]:
print(f"pulled {str(len(combined_2023))} documents from {state} for 2023")

pulled 80 documents from nevada for 2023


## Export

In [51]:
combined_2023.to_csv(f'{state}_2023.csv', index = False)

In [52]:
test = pd.read_csv(f'{state}_2023.csv')
test.head()

,title,link,content,date_clean
0,Governor Lombardo Announces Chief of Staff Tra...,https://gov.nv.gov/Newsroom/PRs/2023/2023-12-2...,"CARSON CITY, NV – Governor Joe Lombardo today ...",2023-12-27
1,Governor Lombardo Announces Las Vegas Stadium ...,https://gov.nv.gov/Newsroom/PRs/2023/2023-12-0...,"Today, Governor Joe Lombardo announced his app...",2023-12-05
2,Governor Lombardo Announces James Humm as Exec...,https://gov.nv.gov/Newsroom/PRs/2023/2023-12-0...,"Today, Governor Joe Lombardo announced James “...",2023-12-05
3,Governor Lombardo Lowers Flags to Half-Staff i...,https://gov.nv.gov/Newsroom/PRs/2023/2023-11-3...,"Today, Governor Joe Lombardo announced that th...",2023-11-30
4,Governor Lombardo Statement on Nevada State Tr...,https://gov.nv.gov/Newsroom/PRs/2023/2023-11-3...,Following the tragic report from Nevada State ...,2023-11-30
